In [48]:
#!pip install ../input/mtcnn-package/mtcnn-0.1.0-py3-none-any.whl

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.patches import Rectangle # For drawing rectangle around faces

import os
import sys

import keras
from keras import Model, Sequential
from keras.layers import *
from keras.optimizers import *

import torch

from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss

import cv2
from mtcnn import MTCNN

from tqdm.notebook import tqdm
import random

import warnings
warnings.filterwarnings("ignore")

First we define our directory paths and directory lists - including the directory where we will save our train and test images that we extract from the videos. 

In [123]:
train_sample_videos_path = '../input/train_sample_videos/'
test_videos_path = '../input/test_videos/'
train_sample_images_path = "../input/train_sample_images/" # path to save train images to
test_images_path = "../input/test_images/" # path to save test images to

test_videos = os.listdir(test_videos_path) # List test vids

Now we loop round all the videos in our directory to extract images for each video.

**TO DO:**

 - **Be able to define the number of frames to extract, and where these frames are (e.g. frame 1,5,10,... or randomly select frame numbers)**
 - **Have file path as an arg so we can define train or test file paths**

In [68]:
def extract_faces(videos_dir_path, images_dir_path, frames=1):
    """
    Inputs a directory of videos, extracts n frames. 
    Outputs images of ANY faces detected in those frames.
    
    videos_dir_path: (str) Path to your directory of videos
    images_dir_path: (str) Path to where you'll save your images to
    frames: (int or list) Number of frames. If int, take that many 
            frames. If list, take frame numbers specified in list. 
    """
    device = 'CUDA:0' if torch.cuda.is_available() else 'CPU'
    print(f'Running on device: {device}')

    videos_dir = os.listdir(videos_dir_path) # List train vids
    
    # Extract images from videos
    if type(frames) == list:
        print(f'Extracting frames {frames} from videos')
    else:
        print(f'Extracting {frames} random frames from videos')
        
    with tqdm(total=len(range(0,5))) as pbar: # Edited out len(videos_dir) for dev
        for i in range(0, 5): # Edited out len(videos_dir) for dev
            file_name = videos_dir[i] # file name with .ext
            vid_name = file_name.split('.')[0] # file name without .ext
            file_path = videos_dir_path + file_name # full file path

            if type(frames) == list:
                for num in range(0, len(frames)):
                    cap = cv2.VideoCapture(file_path)
                    total_frames = cap.get(7)
                    vid_length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
                    cap.set(1, num) # EDIT HERE FOR FRAME NUMBER
                    ret, frame = cap.read()
                    image_name = vid_name + '_' + str(num) + '.jpg'
                    cv2.imwrite(os.path.join(train_sample_images_path, image_name), frame) # Save frame as image
                    cv2.destroyAllWindows()
                    cap.release()
            else:
                for num in range(0, frames):
                    cap = cv2.VideoCapture(file_path)
                    total_frames = cap.get(7)
                    vid_length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
                    cap.set(1, random.randint(0, vid_length)) # EDIT HERE FOR FRAME NUMBER
                    ret, frame = cap.read()
                    image_name = vid_name + '_' + str(num) + '.jpg'
                    cv2.imwrite(os.path.join(train_sample_images_path, image_name), frame) # Save frame as image
                    cv2.destroyAllWindows()
                    cap.release()
            pbar.update(1)
        
        
    images_dir = os.listdir(images_dir_path) # List newly created training images
    detector = MTCNN()
    
    print('Extracting faces from frames')
    with tqdm(total=len(images_dir)) as pbar:
        for image in range(0, len(images_dir)):
            image_name = images_dir[image].split('.')[0] # Get image name without .ext

            # Read image and detect faces
            frame = cv2.imread(images_dir_path + images_dir[image])
            result = detector.detect_faces(frame)

            # Extract and save faces as their own images
            for face in range(0, len(result)):
                # Only extract the face if confidence is more than or equal to 0.97
                if result[face]['confidence'] >= 0.97:            
                    startX, startY, endX, endY = result[face]['box'] # Get box coordinates
                    crop_img = frame[startY:startY + endY, startX:startX + endX] # extract just the face
                    cv2.imwrite(images_dir_path + image_name + '_' + str(face) + '.jpg', crop_img)
            os.remove(images_dir_path + images_dir[image]) # Delete original image
            pbar.update(1)

In [69]:
extract_faces(train_sample_videos_path, train_sample_images_path, frames=2)

Running on device: CPU
Extracting 2 random frames from videos



Extracting faces from frames


In [131]:
train_sample_images = os.listdir('../input/train_sample_images/')

In [139]:
X = []
for img in tqdm(train_sample_images):
    img = train_sample_images_path + img
    img = cv2.cvtColor(cv2.imread(img), cv2.COLOR_BGR2RGB)
    X.append(img)

In [ ]:
X_val = 

In [76]:
import json

with open('../input/train_sample_videos/metadata.json') as json_file:
    data = json.load(json_file)
data

{'aagfhgtpmv.mp4': {'label': 'FAKE',
  'split': 'train',
  'original': 'vudstovrck.mp4'},
 'aapnvogymq.mp4': {'label': 'FAKE',
  'split': 'train',
  'original': 'jdubbvfswz.mp4'},
 'abarnvbtwb.mp4': {'label': 'REAL', 'split': 'train', 'original': None},
 'abofeumbvv.mp4': {'label': 'FAKE',
  'split': 'train',
  'original': 'atvmxvwyns.mp4'},
 'abqwwspghj.mp4': {'label': 'FAKE',
  'split': 'train',
  'original': 'qzimuostzz.mp4'},
 'acifjvzvpm.mp4': {'label': 'FAKE',
  'split': 'train',
  'original': 'kbvibjhfzo.mp4'},
 'acqfdwsrhi.mp4': {'label': 'FAKE',
  'split': 'train',
  'original': 'ccfoszqabv.mp4'},
 'acxnxvbsxk.mp4': {'label': 'FAKE',
  'split': 'train',
  'original': 'fjlyaizcwc.mp4'},
 'acxwigylke.mp4': {'label': 'FAKE',
  'split': 'train',
  'original': 'ffcwhpnpuw.mp4'},
 'aczrgyricp.mp4': {'label': 'FAKE',
  'split': 'train',
  'original': 'slwkmefgde.mp4'},
 'adhsbajydo.mp4': {'label': 'FAKE',
  'split': 'train',
  'original': 'fysyrqfguw.mp4'},
 'adohikbdaz.mp4': {'label

In [119]:
train_sample_images = os.listdir(train_sample_images_path)
train_sample_images = list(map(lambda x: x.split('_')[0], train_sample_images))

for image in train_sample_images:
    print(image + ' : ' + data[image + '.mp4']['label'])

aagfhgtpmv : FAKE
aagfhgtpmv : FAKE
aapnvogymq : FAKE
aapnvogymq : FAKE
aapnvogymq : FAKE
abarnvbtwb : REAL
abarnvbtwb : REAL
abqwwspghj : FAKE
abqwwspghj : FAKE


In [109]:
if 
    data['abqwwspghj.mp4']

{'label': 'FAKE', 'split': 'train', 'original': 'qzimuostzz.mp4'}